In [1]:
!pip install accelerate peft bitsandbytes transformers==4.45.2 sentence-transformers==3.1.1 trl==0.11 wandb torch python-dotenv pyyaml tensorflow huggingface-hub openai

  Using cached peft-0.14.0-py3-none-any.whl.metadata (13 kB)
  Using cached bitsandbytes-0.45.3-py3-none-manylinux_2_24_x86_64.whl.metadata (5.0 kB)
  Using cached transformers-4.45.2-py3-none-any.whl.metadata (44 kB)
  Using cached sentence_transformers-3.1.1-py3-none-any.whl.metadata (10 kB)
  Using cached trl-0.11.0-py3-none-any.whl.metadata (12 kB)
  Using cached wandb-0.19.7-py3-none-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (10 kB)
  Using cached tokenizers-0.20.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
  Using cached tyro-0.9.16-py3-none-any.whl.metadata (9.4 kB)
  Using cached docker_pycreds-0.4.0-py2.py3-none-any.whl.metadata (1.8 kB)
  Using cached sentry_sdk-2.22.0-py2.py3-none-any.whl.metadata (10 kB)
  Using cached docstring_parser-0.16-py3-none-any.whl.metadata (3.0 kB)
  Using cached shtab-1.7.1-py3-none-any.whl.metadata (7.3 kB)
  Using cached typeguard-4.4.2-py3-none-any.whl.metadata (3.8 kB)
Using cached transformers-

In [2]:
!pip install --upgrade  --force-reinstall -q git+https://github.com/lucapodo/pip-newton.git

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-multimodal 1.1.1 requires nvidia-ml-py3==7.352.0, which is not installed.
dash 2.18.1 requires dash-core-components==2.0.0, which is not installed.
dash 2.18.1 requires dash-html-components==2.0.0, which is not installed.
dash 2.18.1 requires dash-table==5.0.0, which is not installed.
jupyter-ai 2.28.3 requires faiss-cpu!=1.8.0.post0,<2.0.0,>=1.8.0, which is not installed.
autogluon-core 1.1.1 requires scikit-learn<1.4.1,>=1.3.0, but you have scikit-learn 1.5.2 which is incompatible.
autogluon-core 1.1.1 requires scipy<1.13,>=1.5.4, but you have scipy 1.14.1 which is incompatible.
autogluon-multimodal 1.1.1 requires jsonschema<4.22,>=4.18, but you have jsonschema 4.23.0 which is incompatible.
autogluon-multimodal 1.1.1 requires omegaconf<2.3.0,>=2.1.1, but you have omegaconf 2.3.0 which is incompatible.


In [4]:
!pip install accelerate==0.27.2

  Using cached accelerate-0.27.2-py3-none-any.whl.metadata (18 kB)
Using cached accelerate-0.27.2-py3-none-any.whl (279 kB)
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.21.0
    Uninstalling accelerate-0.21.0:
      Successfully uninstalled accelerate-0.21.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-multimodal 1.1.1 requires nvidia-ml-py3==7.352.0, which is not installed.
autogluon-multimodal 1.1.1 requires accelerate<0.22.0,>=0.21.0, but you have accelerate 0.27.2 which is incompatible.
autogluon-multimodal 1.1.1 requires jsonschema<4.22,>=4.18, but you have jsonschema 4.23.0 which is incompatible.
autogluon-multimodal 1.1.1 requires omegaconf<2.3.0,>=2.1.1, but you have omegaconf 2.3.0 which is incompatible.
autogluon-multimodal 1.1.1 requires scikit-learn<1.4.1,>=1.3.0, but you have scikit-learn 1.5.2 which

In [7]:
# 0. imports
import torch
from datasets import load_dataset
from accelerate import Accelerator
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
    EarlyStoppingCallback, 
    IntervalStrategy
)
from peft import LoraConfig, PeftModel
from datasets import load_dataset
from dotenv import load_dotenv
from trl import PPOConfig, PPOTrainer, set_seed
import os
import yaml
import torch
import os
from huggingface_hub import login
import os
import wandb
import pandas as pd

load_dotenv("config.env")

with open('config.yaml') as file:
    config= yaml.safe_load(file)
    #print(config['hugginfaces']['model_name'])

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
print("PyTorch version:", torch.__version__)
if torch.cuda.is_available():
    print("CUDA version:", torch.version.cuda)
else:
    print("CUDA is not available.")

login(token='HUGGING_FACE_TOKEN')
os.environ["WANDB_PROJECT"] = "finetuning"
os.environ["WANDB_NAME"]= "finetuning-100"
os.environ["WANDB_API_KEY"] = "WANDB_TOKEN"
os.environ["OPENAI_API_KEY"] = "OPEN_AI_KEY"
wandb.login()
wandb.init(dir=config['cache_dir'])
model_path = 'your-project-name'

2025-03-03 23:01:53.610991: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-03 23:01:53.626847: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-03 23:01:53.631822: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-03 23:01:53.643697: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
wandb: Using wandb-core as the SDK 

PyTorch version: 2.3.1.post300
CUDA version: 12.0


wandb: Currently logged in as: lpodo (deepvizlab) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [8]:
from newtonmetrics.vegazero.VegaZero2VegaLite import VegaZero2VegaLite 
from newtonmetrics.newton.newton import Newton 

n = Newton()
n.NormalizeData(2)

0.4

# Chat GPT Fine-Tuning

In [9]:
org_id = "org-FAk31gp9DFnG4Xmi1xsUEirF"
project_id = "proj_Y04rDKcS5s0SF4yBTAcwA1tf"

In [10]:
from openai import OpenAI

client = OpenAI(
  organization=org_id,
  project=project_id,
)

In [11]:
#!openai api files.create -p fine-tune -f fine-tuning-openai.jsonl

In [12]:
# Replace with your fine-tuned model ID
fine_tuned_model = "FINETUNED_MODEL_ID"

# load the dataset

In [13]:
import torch
import torch.nn as nn

# Define a simple reward model
class SimpleRewardModel(nn.Module):
    def __init__(self):
        super(SimpleRewardModel, self).__init__()

    def forward(self, prompts, responses):
        # Example: Return constant rewards (use real logic in production)
        batch_size = len(responses)
        return torch.ones(batch_size)  # Rewards for each response


In [14]:
def build_dataset(tokenizer):
    ds = load_dataset(dataset_name, split="test", revision="0.0.2",download_mode="force_redownload", cache_dir="./tmp" )
    def remove_reponse(sample):
        sample['hardness'] = sample['text'].split("@")[0].strip()
        text = sample['text'].split('@')[1]
        
        sample['response'] = text.split("### Response:")[1].strip()
        sample['text'] = text.split("### Response:")[0] + "### Response: mark\n"
        return sample

    def remove_reponse_and_intro(sample):
        sample['hardness'] = sample['text'].split("@")[0].strip()
        text = sample['text'].split('@')[1]
        
        sample['response'] = text.split("### Response:")[1].strip()
        text = text.split("### Response:")[0] + "### Response: \nmark"
        sample['text'] = "### Instruction:" + text.split("### Instruction:")[1]
        return sample
        
    ds = ds.map(remove_reponse_and_intro, batched=False)
    
    # Split the dataset into the first 100 instances and the rest
    # first_100_instances = ds.select(range(100))
    # remaining_instances = ds.select(range(100, len(ds)))

    def tokenize(sample):
        sample["input_ids"] = tokenizer.encode(sample["text"], padding = True)
        #sample["query"] = '<s>' + tokenizer.decode(sample["input_ids"]).split('<s>')[2]
        sample["query"] = tokenizer.decode(sample["input_ids"])
        return sample

    ds = ds.map(tokenize, batched=False)
    ds.set_format(type="torch")
    return ds

In [15]:
def can_compile(result, ground):
    res = n.can_compile(result,ground)
    return res

def get_newton_score(result, ground):
    # res = n.compute_score_raff(result, ground)
    res = n.compute_score_100(result,ground)
    # res = n.only_jaacard(result,ground)
    return res

# BERT

In [ ]:
def bert_loss(predicted_reward, discrete_reward, policy_loss, value_loss):
    """
    Calculate the observer model's loss based on the differences in statistics
    between the current and previous steps.
    
    Parameters:
    - current_stats: dict, containing the current statistics from the PPO learner.
    - previous_stats: dict, containing the statistics from the previous step.
    - weights: dict, containing weights for each component of the loss.
    
    Returns:
    - loss: float, the calculated loss for the observer model focusing on changes in performance.
    """
    weights = {
        'pl': -0.5,  # Negative weight if we want to encourage reduction in policy loss
        'vl': -0.5,   # Negative weight if we want to encourage reduction in value loss
        'kld': 0.1,  # Positive or negative based on whether you want to increase or decrease KL divergence
        'en': -0.1,  # Negative if reducing entropy is desired
        'alpha': 1.5,
        'beta': 0.2,
        'epsolon': 0.01
    }
    # print(predicted_reward)
    # print(discrete_reward)
    # predicted_reward = int(predicted_reward)
    # discrete_reward = int(discrete_reward)
    # Calculate components of the loss based on these differences
    reward_diff = abs(predicted_reward - discrete_reward)

    policy_loss_component = weights['pl'] * policy_loss
    value_loss_component = weights['vl'] * value_loss

    metrics = policy_loss_component + value_loss_component
    
    
    reward_benefit =  metrics / (reward_diff + weights['epsolon'])
    
    # print(f"metrics: {reward_benefit}")
    
    loss = weights['alpha'] * reward_diff + weights['beta'] * reward_benefit
    # trial_loss = weights['alpha'] * reward_diff
    # print(f"loss: {loss}")

    return loss


In [ ]:
import torch

# Initialize storage for dataset
training_data = {
    "prediction": [],
    "groundtruth": [],
    "reward": [],
    "policy_loss":[],
    "value_loss":[],
    # "kl_diff":[],
    # "entropy":[]
}

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AdamW

# Load pre-trained TinyBERT and its tokenizer
model_name = 'prajjwal1/bert-tiny'  # Example TinyBERT model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=1)  # Assuming a regression task


# Define the optimizer
optimizer_bert = AdamW(model.parameters(), lr=2e-5)

In [ ]:
from datasets import Dataset, load_dataset
import pandas as pd
from transformers import AutoTokenizer

# Load data
df_train = pd.read_csv("./training_data_weak.csv")

# Convert DataFrame to Hugging Face Dataset
dataset = Dataset.from_pandas(df_train)
tokenizer_1 = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, cache_dir="./tmp")

# Ensure the tokenizer uses the correct padding token
if tokenizer_1.pad_token is None:
    tokenizer_1.pad_token = tokenizer_1.eos_token
    tokenizer_1.model.config.pad_token_id = tokenizer_1.eos_token_id

# Formatting function adapted for your data structure
def formatting_func(examples):
    combined = examples['prediction'] + ' ' + examples['groundtruth']
    encoded = tokenizer_1(combined,
                            padding="max_length",  # Ensure all outputs are padded to the max_length
                            truncation=True,        # Ensure all outputs are truncated to fit the max_length
                            max_length=128,        # Define max_length according to model's capability
                            return_tensors="pt")   # Use numpy to ensure compatibility with Hugging Face Dataset
    
    # print("Shape of input_ids:", encoded['input_ids'].shape)
    # print("Shape of attention_mask:", encoded['attention_mask'].shape)
    
    return {
        "input_ids": encoded["input_ids"][0],  # Remove batch dimension
        "attention_mask": encoded["attention_mask"][0],
        "reward": examples["reward"],
        "policy_loss": examples['policy_loss'],
        "value_loss": examples['value_loss'],
        # "kl_diff": examples['kl_diff'],
        # "entropy": examples['entropy']
    }


# Apply formatting function
formatted_dataset = dataset.map(formatting_func, batched=False)

print(formatted_dataset[0])

formatted_dataset = formatted_dataset.remove_columns(['prediction', 'groundtruth'])


print(formatted_dataset[0])

# print(formatted_dataset[0])

train_test_split = formatted_dataset.train_test_split(test_size=0.1)


In [ ]:
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer_1, return_tensors="pt")
data_loader = DataLoader(formatted_dataset, batch_size=2, collate_fn=data_collator)

for batch in data_loader:
    print("Batch 'input_ids' shape:", batch['input_ids'].shape)
    print("Batch 'attention_mask' shape:", batch['attention_mask'].shape)
    break

In [ ]:
from transformers import TrainingArguments, Trainer
from transformers import AutoModel, AutoConfig
import torch
import torch.nn as nn


class CustomBert(nn.Module):
    def __init__(self, model_name):
        super(CustomBert, self).__init__()
        self.config = AutoConfig.from_pretrained(model_name)
        self.bert = AutoModel.from_pretrained(model_name, config=self.config)
        self.classifier = nn.Sequential(
            nn.Linear(self.config.hidden_size, 1),
            nn.Sigmoid()
        )
        
    def forward(self, input_ids, attention_mask):

        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        
        # we only want to get the hidden states of the [CLS] token
        pooled_output = outputs.pooler_output
        
        # pass it through the custom classifier
        logits = self.classifier(pooled_output)
        
        return logits


# custom trainer method to implement our loss function
class CustomTrainer(Trainer):

    def compute_loss(self, model, inputs, return_outputs=False):
        # Extract all necessary inputs
        labels = inputs.pop("reward")
        policy_loss = inputs.pop("policy_loss")
        value_loss = inputs.pop("value_loss")
        # kl_diff = inputs.pop("kl_diff")
        # entropy = inputs.pop("entropy")
        # print('model inputs: ' + inputs)
        # Forward pass
        outputs = model(**inputs)
        print(f"Output {outputs}")
        logits = outputs
        
        # loss = bert_loss_KL(logits, labels, policy_loss, value_loss, kl_diff, entropy)
        loss = bert_loss(logits, labels, policy_loss, value_loss)
        loss = torch.mean(loss)
        if return_outputs:
            return (loss, outputs)
        else:
            return loss

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)

# text = "I am very happy today!"
# tokens = tokenizer(text, return_tensors='pt', padding='max_length', max_length=128)

model = CustomBert(model_name)
for param in model.parameters(): param.data = param.data.contiguous()


In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding


# Configuring the training arguments
training_args = TrainingArguments(
    output_dir="./models/bert_reward_model_v5",
    per_device_train_batch_size=1,
    evaluation_strategy="no",
    logging_steps=1,
    num_train_epochs = 10,
    report_to=None,
    remove_unused_columns=False,
    save_safetensors=False
)

# use new trainer
trainer = CustomTrainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    train_dataset=train_test_split['train']
)

trainer.train()

# two batch BERT

In [13]:
import torch

def bert_loss_two(predicted_rewards, discrete_rewards, policy_loss_change, value_loss_change):
    """
    Calculate the observer model's loss for a batch of two samples.
    
    Parameters:
    - predicted_rewards: Tensor of shape (2,), predicted rewards for both instances in the batch.
    - discrete_rewards: Tensor of shape (2,), discrete rewards for both instances.
    - policy_loss_change: Float, the change in policy loss from the batch.
    - value_loss_change: Float, the change in value loss from the batch.
    
    Returns:
    - loss: float, the calculated loss considering both individual reward accuracy and overall policy impact.
    """
    
    weights = {
        'pl': -0.5,  # Negative weight if we want to encourage reduction in policy loss
        'vl': -0.5,  # Negative weight if we want to encourage reduction in value loss
        'alpha': 2,  # Weight for reward difference loss
        'beta': 0.5,   # Weight for policy/value loss impact
        'epsilon': 0.01  # Small value to prevent division by zero
    }

    # Ensure inputs are tensors
    predicted_rewards = torch.tensor(predicted_rewards, dtype=torch.float32)
    discrete_rewards = torch.tensor(discrete_rewards, dtype=torch.float32)

    # Compute reward differences for both instances
    reward_diff = torch.abs(predicted_rewards - discrete_rewards)  # Shape (2,)

    # Compute the impact of the policy and value loss changes on the reward adjustment
    policy_loss_component = weights['pl'] * policy_loss_change  # Single value
    value_loss_component = weights['vl'] * value_loss_change  # Single value
    
    # Batch-level effect on rewards
    metrics = policy_loss_component + value_loss_component  # Scalar

    # Reward benefit calculation for each instance
    reward_benefit = metrics / (reward_diff + weights['epsilon'])  # Shape (2,)

    # Compute loss for each instance in batch
    instance_losses = weights['alpha'] * reward_diff + weights['beta'] * reward_benefit  # Shape (2,)

    # Aggregate total loss (mean loss across batch)
    total_loss = torch.mean(instance_losses)

    return total_loss


In [14]:
import torch

# Initialize storage for dataset
training_data = {
    "prediction 1": [],
    "groundtruth 1": [],
    "reward 1": [],
    "prediction 2": [],
    "groundtruth 2": [],
    "reward 2": [],
    "policy_loss":[],
    "value_loss":[],
    # "kl_diff":[],
    # "entropy":[]
}

In [15]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AdamW

# Load pre-trained TinyBERT and its tokenizer
model_name = 'prajjwal1/bert-tiny'  # Example TinyBERT model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=1)  # Assuming a regression task


# Define the optimizer
optimizer_bert = AdamW(model.parameters(), lr=2e-5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.11/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [16]:
from datasets import Dataset, load_dataset
import pandas as pd
from transformers import AutoTokenizer

# Load data
df_train = pd.read_csv("./training_data_weak.csv")

# Convert DataFrame to Hugging Face Dataset
dataset = Dataset.from_pandas(df_train)
tokenizer_1 = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, cache_dir="./tmp")

# Ensure the tokenizer uses the correct padding token
if tokenizer_1.pad_token is None:
    tokenizer_1.pad_token = tokenizer_1.eos_token
    tokenizer_1.model.config.pad_token_id = tokenizer_1.eos_token_id

# Formatting function adapted for your data structure
# def formatting_func(examples):
#     combined = examples['prediction'] + ' ' + examples['groundtruth']
#     encoded = tokenizer_1(combined,
#                             padding="max_length",  # Ensure all outputs are padded to the max_length
#                             truncation=True,        # Ensure all outputs are truncated to fit the max_length
#                             max_length=128,        # Define max_length according to model's capability
#                             return_tensors="pt")   # Use numpy to ensure compatibility with Hugging Face Dataset
    
#     # print("Shape of input_ids:", encoded['input_ids'].shape)
#     # print("Shape of attention_mask:", encoded['attention_mask'].shape)
    
#     return {
#         "input_ids": encoded["input_ids"][0],  # Remove batch dimension
#         "attention_mask": encoded["attention_mask"][0],
#         "reward": examples["reward"],
#         "policy_loss": examples['policy_loss'],
#         "value_loss": examples['value_loss'],
#         # "kl_diff": examples['kl_diff'],
#         # "entropy": examples['entropy']
#     }

# Formatting function adapted for your data structure
def formatting_func(examples):
    """
    Formats input for training TinyBERT with batched pairs of instances.
    Ensures `input_ids` and `attention_mask` are properly structured.
    """
    combined_1 = examples['prediction 1'] + ' ' + examples['groundtruth 1']
    combined_2 = examples['prediction 2'] + ' ' + examples['groundtruth 2']

    # Tokenize both instances in the batch
    encoded_1 = tokenizer_1(combined_1, padding="max_length", truncation=True, max_length=128)
    encoded_2 = tokenizer_1(combined_2, padding="max_length", truncation=True, max_length=128)

    return {
        "input_ids": [encoded_1["input_ids"], encoded_2["input_ids"]],  # Stacked inputs
        "attention_mask": [encoded_1["attention_mask"], encoded_2["attention_mask"]],  # Stacked attention masks
        "reward 1": examples["reward 1"],
        "reward 2": examples["reward 2"],
        "policy_loss": examples["policy_loss"],
        "value_loss": examples["value_loss"]
    }


# Apply formatting function
formatted_dataset = dataset.map(formatting_func, batched=False)

print(formatted_dataset[0])

formatted_dataset = formatted_dataset.remove_columns(['prediction 1', 'groundtruth 1', 'prediction 2', 'groundtruth 2'])

print(formatted_dataset[0])

train_test_split = formatted_dataset.train_test_split(test_size=0.1)


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

{'prediction 1': ' bar data wine encoding x grape y aggregate count grape transform group x sort x asc\n### Explanation:\n\nBar chart to show the count of grapes by different grapes, the x-axis is the grapes and the y-axis the the total of each grapes.\nThe', 'groundtruth 1': 'mark bar data wine encoding x grape y aggregate count grape transform group x sort x asc', 'reward 1': 0.703125, 'prediction 2': ' bar data swimmer encoding x meter y aggregate none id transform group x sort x asc\n### Explanation:\n\nThe instruction asks to show the encoding of the meter by a x axis and y axis, and group by x in the bar mark. The aggregate', 'groundtruth 2': 'mark bar data swimmer encoding x meter_300 y aggregate none id transform sort x asc', 'reward 2': 0.6875, 'policy_loss': 0.0031077964, 'value_loss': -0.0006714761, 'input_ids': [[101, 3347, 2951, 4511, 17181, 1060, 14722, 1061, 9572, 4175, 14722, 10938, 2177, 1060, 4066, 1060, 2004, 2278, 1001, 1001, 1001, 7526, 1024, 3347, 3673, 2000, 2265

In [17]:
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer_1, return_tensors="pt")
data_loader = DataLoader(formatted_dataset, batch_size=2, collate_fn=data_collator)

for batch in data_loader:
    print("Batch 'input_ids' shape:", batch['input_ids'].shape)
    print("Batch 'attention_mask' shape:", batch['attention_mask'].shape)
    break

Batch 'input_ids' shape: torch.Size([2, 2, 128])
Batch 'attention_mask' shape: torch.Size([2, 2, 128])


In [18]:
from transformers import TrainingArguments, Trainer
from transformers import AutoModel, AutoConfig
import torch
import torch.nn as nn


import torch
import torch.nn as nn
from transformers import AutoModel, AutoConfig

class CustomBert(nn.Module):
    def __init__(self, model_name):
        super(CustomBert, self).__init__()
        self.config = AutoConfig.from_pretrained(model_name)
        self.bert = AutoModel.from_pretrained(model_name, config=self.config)
        # Two reward outputs (one per instance in batch)
        self.classifier = nn.Sequential(
            nn.Linear(self.config.hidden_size, 1), 
            nn.Tanh()
        )
        
    def forward(self, input_ids, attention_mask):
        """
        Forward pass for two instances per batch.
        `input_ids` and `attention_mask` should have shape (batch_size, 2, sequence_length)
        """
        batch_size, num_instances, seq_length = input_ids.shape  # Expecting shape (B, 2, L)
        
        # Flatten batch for processing in BERT
        input_ids = input_ids.view(batch_size * num_instances, seq_length)  # Shape: (B * 2, L)
        attention_mask = attention_mask.view(batch_size * num_instances, seq_length)  # Shape: (B * 2, L)

        # Pass through BERT
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output  # Shape: (B * 2, hidden_size)

        # Predict rewards
        logits = self.classifier(pooled_output)  # Shape: (B * 2, 1)
        reward_prediction = (logits + 1) / 2  # Scale from [-1, 1] to [0, 1]
        # Reshape back to (B, 2)
        return logits.view(batch_size, num_instances)



# custom trainer method to implement our loss function
class CustomTrainer(Trainer):

    def compute_loss(self, model, inputs, return_outputs=False):
        """
        Computes loss for two instances per batch.
        """
        input_ids = inputs["input_ids"]  # Shape: (batch_size, 2, sequence_length)
        attention_mask = inputs["attention_mask"]  # Shape: (batch_size, 2, sequence_length))
        labels = torch.stack([inputs.pop("reward 1"), inputs.pop("reward 2")], dim=1).to(torch.float32) # Shape: (batch_size, 2)
        policy_loss_change = torch.tensor(inputs["policy_loss"], dtype=torch.float32, requires_grad=True)
        value_loss_change = torch.tensor(inputs["value_loss"], dtype=torch.float32, requires_grad=True)
        # Forward pass (new batch format)
        outputs = model(input_ids, attention_mask)  # Shape: (batch_size, 2)
        # Compute loss
        loss = bert_loss_two(outputs, labels, policy_loss_change, value_loss_change)  
        loss = torch.mean(loss)  # Ensure it's a scalar
        if return_outputs:
            return (loss, outputs)
        else:
            return loss

    # def compute_loss(self, model, inputs, return_outputs=False):
    #     # Extract all necessary inputs
    #     labels = inputs.pop("reward")
    #     policy_loss = inputs.pop("policy_loss")
    #     value_loss = inputs.pop("value_loss")
    #     # kl_diff = inputs.pop("kl_diff")
    #     # entropy = inputs.pop("entropy")
    #     # print('model inputs: ' + inputs)
    #     # Forward pass
    #     outputs = model(**inputs)
    #     print(f"Output {outputs}")
    #     logits = outputs
        
    #     # loss = bert_loss_KL(logits, labels, policy_loss, value_loss, kl_diff, entropy)
    #     loss = bert_loss(logits, labels, policy_loss, value_loss)
    #     loss = torch.mean(loss)
    #     if return_outputs:
    #         return (loss, outputs)
    #     else:
    #         return loss

In [19]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)

# text = "I am very happy today!"
# tokens = tokenizer(text, return_tensors='pt', padding='max_length', max_length=128)

model = CustomBert(model_name)
for param in model.parameters(): param.data = param.data.contiguous()

In [20]:
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding


# Configuring the training arguments
training_args = TrainingArguments(
    output_dir="./models/bert_reward_model_v5",
    per_device_train_batch_size=1,
    evaluation_strategy="no",
    logging_steps=1,
    num_train_epochs = 20,
    report_to=None,
    remove_unused_columns=False,
    save_safetensors=False
)

# use new trainer
trainer = CustomTrainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    train_dataset=train_test_split['train']
)

trainer.train()

/opt/conda/lib/python3.11/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
/tmp/ipykernel_1778/382386652.py:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  policy_loss_change = torch.tensor(inputs["policy_loss"], dtype=torch.float32, requires_grad=True)
/tmp/ipykernel_1778/382386652.py:56: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourc

Step,Training Loss
1,0.599200
2,0.578900
3,0.690400
4,0.790700
5,0.351300
6,0.856200
7,1.023400
8,0.488800
9,0.625100
10,0.632400


/tmp/ipykernel_1778/382386652.py:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  policy_loss_change = torch.tensor(inputs["policy_loss"], dtype=torch.float32, requires_grad=True)
/tmp/ipykernel_1778/382386652.py:56: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  value_loss_change = torch.tensor(inputs["value_loss"], dtype=torch.float32, requires_grad=True)
/tmp/ipykernel_1778/2989015306.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  predicted_rewards = torch.tensor(predicted_rewards, dtype=torch.float32)
/tmp/ipykernel_1778/2989015

TrainOutput(global_step=3600, training_loss=0.6762000626491176, metrics={'train_runtime': 24.648, 'train_samples_per_second': 146.057, 'train_steps_per_second': 146.057, 'total_flos': 0.0, 'train_loss': 0.6762000626491176, 'epoch': 20.0})

# SON TRAINING

In [10]:
from trl import AutoModelForCausalLMWithValueHead, create_reference_model
from transformers import AutoTokenizer, GenerationConfig

In [16]:
#### run again
dataset_name = "LucaPodo/newton-dataset-v1"
dataset_version = "1.0"
model_name = "./models/200-easy"
# model_name = "DeepvizLab/newton-7b-1k"
################################################################################
# QLoRA parameters
################################################################################
lora_r = 128
lora_alpha = 16 #16
lora_dropout = 0.1
################################################################################
# bitsandbytes parameters
################################################################################
use_4bit = True
bnb_4bit_compute_dtype = "bfloat16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False
################################################################################
# TrainingArguments parameters
################################################################################
output_dir = "/content/drive/MyDrive/DeepvizLab/NewtonLLM/llama2"
num_train_epochs = 1
fp16 = False
bf16 = False
per_device_train_batch_size = 4
per_device_eval_batch_size = 4
gradient_accumulation_steps = 1
gradient_checkpointing = True
max_grad_norm = 0.3
learning_rate = 2e-5
weight_decay = 0.001
optim = "paged_adamw_32bit"
lr_scheduler_type = "cosine"
max_steps = -1
warmup_ratio = 0.03
group_by_length = True
save_steps = 0
logging_steps = 25
################################################################################
# SFT parameters
################################################################################
max_seq_length = None
packing = False
device_map = {"": 0}
batch_size = 2

In [16]:
config = PPOConfig(
    model_name=model_path,
    learning_rate=learning_rate,#1.41e-5,
    mini_batch_size=1,
    batch_size=2,
    gradient_accumulation_steps=1,
    early_stopping=True,
    target_kl=1,
    kl_penalty="kl",
    seed=0,
    log_with='wandb',
    #callbacks=[WandbCallback(log=False)]
)

set_seed(config.seed)

/opt/conda/lib/python3.11/site-packages/trl/trainer/ppo_config.py:207: FutureWarning: `PPOConfig` is deprecated and will be removed in the future. Please use `PPOv2Config` with `PPOv2Trainer` instead.
  warnings.warn(


In [17]:
# Reload model in FP16 and merge it with LoRA weights
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit= True,
#     bnb_4bit_compute_dtype= "bfloat16",
#     bnb_4bit_quant_type= "nf4",
#     bnb_4bit_use_double_quant= False,
# )

model_ = AutoModelForCausalLM.from_pretrained(
    "./models/200-easy-full",
    low_cpu_mem_usage=True,
    return_dict=True,
    # quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    device_map={"": 0},
)

tokenizer_ = AutoTokenizer.from_pretrained("./models/200-easy-full-tokenizer", trust_remote_code=True)

tokenizer_.padding_side = "left"
model_.resize_token_embeddings(len(tokenizer_))
model_.config.pad_token_id = tokenizer_.pad_token_id

# model_ = PeftModel.from_pretrained(base_model, "./models/200-easy")
# model_ = model_.merge_and_unload()

print("BOS Token:", tokenizer_.bos_token_id)
print("EOS Token:", tokenizer_.eos_token_id)
print("PAD Token:", tokenizer_.pad_token_id)
print(tokenizer_.all_special_tokens)
assert model_.config.pad_token_id == tokenizer_.pad_token_id, "The model's pad token ID does not match the tokenizer's pad token ID!"

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

BOS Token: 128000
EOS Token: 128001
PAD Token: 128256
['<|begin_of_text|>', '<|end_of_text|>', '[PAD]']


In [18]:
# model_.save_pretrained("./models/200-easy-merged")
# tokenizer_.save_pretrained("./models/200-easy-merged-token")

In [19]:
# ## Load the model in 4-bit quantization
# device_map = {"": 0}
# #compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
# bnb_config = BitsAndBytesConfig(
#     use_4bit= True,
#     bnb_4bit_compute_dtype= "bfloat16",
#     bnb_4bit_quant_type= "nf4",
#     use_nested_quant= False,
# )

# # Load base model
# model_ = AutoModelForCausalLM.from_pretrained(
#     "./models/200-easy-merged",
#     quantization_config=bnb_config,
#     device_map=device_map,
#     torch_dtype=torch.bfloat16,#torch.bfloat16,
#     cache_dir="./tmp"
# )
# model_.config.use_cache = False
# model_.config.pretraining_tp = 1

# tokenizer_ = AutoTokenizer.from_pretrained("./models/200-easy-merged-token", trust_remote_code=True, cache_dir="./tmp") #nel caso rimuovere torch_dtype torch.bfloat16
# model_.resize_token_embeddings(len(tokenizer_))
# model_.config.pad_token_id = tokenizer_.pad_token_id

# assert model_.config.pad_token_id == tokenizer_.pad_token_id, "The model's pad token ID does not match the tokenizer's pad token ID!"

# tokenizer_.padding_side = "left"
# print(tokenizer_.all_special_tokens)

In [20]:
# dataset_location= "LucaPodo/newton-dataset-v1"
# test = load_dataset(dataset_location, split="test", revision="4.0.4", cache_dir = "./tmp")
# pipe = pipeline(task="text-generation", model=model_, tokenizer=tokenizer_, max_length=400, eos_token_id=tokenizer_.eos_token_id)

# i = 100
# prompt = test[i]['text'].split("### Response:")[0] + "### Response:\nmark"
# print(prompt + '\n')
# groundtruth = test[i]['text'].split("### Response:")[1].strip()
# print(groundtruth + '\n')
# result = pipe(prompt)
# print(result[0]['generated_text'])

In [21]:
from accelerate import Accelerator
from peft import LoraConfig, PeftModel, TaskType
current_device = Accelerator().local_process_index
#tokenizer_ = AutoTokenizer.from_pretrained("./models/200-easy-merged-token", trust_remote_code=True)

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)

ppo_model = AutoModelForCausalLMWithValueHead.from_pretrained(
    model_,
    #torch_dtype=torch.bfloat16, #torch.bfloat16,
    device_map={"": current_device},
    peft_config=peft_config,
)
#model_ref = create_reference_model(ppo_model)

In [22]:
dataset = build_dataset(tokenizer_)

(…)-00000-of-00001-653f5eb8e3f7e385.parquet:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

(…)-00000-of-00001-51e5da44a405016d.parquet:   0%|          | 0.00/349k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/12570 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2455 [00:00<?, ? examples/s]

Map:   0%|          | 0/2455 [00:00<?, ? examples/s]

Map:   0%|          | 0/2455 [00:00<?, ? examples/s]

In [23]:
print(dataset[1])
print(dataset[2])
print(dataset[3])

{'text': "### Instruction:\nWhen do all the researcher role staff start to work ? Bin the time into YEAR interval and count them with a line chart , and order from low to high by the date_from please .\n### Input:\n[('staff_id', 'categorical'), ('project_id', 'numeric'), ('role_code', 'categorical'), ('date_from', 'temporal'), ('date_to', 'temporal'), ('other_details', 'categorical')]\n### Response: \nmark", 'hardness': 'Medium', 'response': 'mark line data project_staff encoding x date_from y aggregate count date_from transform sort x asc bin x by year', 'input_ids': tensor([128000,  14711,  30151,    512,   4599,    656,    682,    279,  32185,
          3560,   5687,   1212,    311,    990,    949,  30444,    279,    892,
          1139,  52005,  10074,    323,   1797,   1124,    449,    264,   1584,
          9676,   1174,    323,   2015,    505,   3428,    311,   1579,    555,
           279,   2457,   5791,   4587,  16853,  14711,   5688,    512,     58,
           493,  28221,  

In [24]:
train = dataset.filter(lambda ds: ds['hardness'] == "Easy" or ds['hardness'] == "Medium")
test = dataset.filter(lambda ds: ds['hardness'] == "Extra Hard" or ds['hardness'] == "Extra Hard")

Filter:   0%|          | 0/2455 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2455 [00:00<?, ? examples/s]

In [25]:
len(train)

1863

In [26]:
import pandas as pd
df = pd.DataFrame(train)

In [27]:


def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])

ppo_trainer = PPOTrainer(
    config,
    ppo_model,
    ref_model=ppo_model,
    tokenizer=tokenizer_,
    dataset=train,
    data_collator=collator,
    optimizer=None,
)

generation_kwargs = {
    "min_length": -1,
    "top_k": 20,
    "top_p": 0.95,
    "do_sample": True,
    "pad_token_id": tokenizer_.pad_token_id,
    "eos_token_id": tokenizer_.eos_token_id,
    "no_repeat_ngram_size": 2,
    "max_new_tokens": 50,
}

/opt/conda/lib/python3.11/site-packages/trl/trainer/ppo_trainer.py:193: FutureWarning: `PPOTrainer` is deprecated and will be removed in trl v0.12. Please use `PPOv2Trainer` instead.
  warnings.warn(


# Load Bert

In [28]:
from transformers import TrainingArguments, Trainer
from transformers import AutoModel, AutoConfig
import torch
import torch.nn as nn


import torch
import torch.nn as nn
from transformers import AutoModel, AutoConfig

class CustomBert(nn.Module):
    def __init__(self, model_name):
        super(CustomBert, self).__init__()
        self.config = AutoConfig.from_pretrained(model_name)
        self.bert = AutoModel.from_pretrained(model_name, config=self.config)
        # Two reward outputs (one per instance in batch)
        self.classifier = nn.Sequential(
            nn.Linear(self.config.hidden_size, 1), 
            nn.Sigmoid()
        )
        
    def forward(self, input_ids, attention_mask):
        """
        Forward pass for two instances per batch.
        `input_ids` and `attention_mask` should have shape (batch_size, 2, sequence_length)
        """
        batch_size, num_instances, seq_length = input_ids.shape  # Expecting shape (B, 2, L)
        
        # Flatten batch for processing in BERT
        input_ids = input_ids.view(batch_size * num_instances, seq_length)  # Shape: (B * 2, L)
        attention_mask = attention_mask.view(batch_size * num_instances, seq_length)  # Shape: (B * 2, L)

        # Pass through BERT
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output  # Shape: (B * 2, hidden_size)

        # Predict rewards
        logits = self.classifier(pooled_output)  # Shape: (B * 2, 1)

        # Reshape back to (B, 2)
        return logits.view(batch_size, num_instances)



# custom trainer method to implement our loss function
class CustomTrainer(Trainer):

    def compute_loss(self, model, inputs, return_outputs=False):
        """
        Computes loss for two instances per batch.
        """
        input_ids = inputs["input_ids"]  # Shape: (batch_size, 2, sequence_length)
        attention_mask = inputs["attention_mask"]  # Shape: (batch_size, 2, sequence_length))
        labels = torch.stack([inputs.pop("reward 1"), inputs.pop("reward 2")], dim=1).to(torch.float32) # Shape: (batch_size, 2)
        policy_loss_change = torch.tensor(inputs["policy_loss"], dtype=torch.float32, requires_grad=True)
        value_loss_change = torch.tensor(inputs["value_loss"], dtype=torch.float32, requires_grad=True)
        # Forward pass (new batch format)
        outputs = model(input_ids, attention_mask)  # Shape: (batch_size, 2)
        # Compute loss
        loss = bert_loss_two(outputs, labels, policy_loss_change, value_loss_change)  
        loss = torch.mean(loss)  # Ensure it's a scalar
        if return_outputs:
            return (loss, outputs)
        else:
            return loss

In [29]:
from transformers import AutoTokenizer

# Assuming CustomBert is defined as shown earlier

model_directory = "./models/bert_reward_model_v5/checkpoint-3600/pytorch_model.bin"  # Adjust path as necessary
tokenizer = AutoTokenizer.from_pretrained("prajjwal1/bert-tiny")

# Load the model using the CustomBert class
model = CustomBert("prajjwal1/bert-tiny")
model.load_state_dict(torch.load(model_directory))

def prepare_inputs(prediction_1, groundtruth_1, prediction_2, groundtruth_2):
    """
    Tokenizes and prepares inputs for the trained TinyBERT model.
    """
    combined_1 = prediction_1 + " " + groundtruth_1
    combined_2 = prediction_2 + " " + groundtruth_2

    encoded_1 = tokenizer(combined_1, padding="max_length", truncation=True, max_length=128, return_tensors="pt")
    encoded_2 = tokenizer(combined_2, padding="max_length", truncation=True, max_length=128, return_tensors="pt")

    input_ids = torch.cat([encoded_1["input_ids"], encoded_2["input_ids"]], dim=0).unsqueeze(0)  # Shape: (1, 2, 128)
    attention_mask = torch.cat([encoded_1["attention_mask"], encoded_2["attention_mask"]], dim=0).unsqueeze(0)  # Shape: (1, 2, 128)

    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask
    }

def make_prediction(prediction_1, groundtruth_1, prediction_2, groundtruth_2):
    """
    Runs inference on two predictions simultaneously.
    """
    inputs = prepare_inputs(prediction_1, groundtruth_1, prediction_2, groundtruth_2)

    model.eval()  # Set model to evaluation mode
    with torch.no_grad():  # Disable gradient computation for inference
        logits = model(**inputs)  # Expected output: (1, 2)

    predictions = logits.squeeze().tolist()  # Convert to list

    return predictions  # Returns [reward_1, reward_2]


# def prepare_input(query, response):
#     combined = query + ' ' + response
#     encoded = tokenizer(combined,
#                         padding="max_length",  # Ensure all outputs are padded to the max_length
#                         truncation=True,        # Ensure all outputs are truncated to fit the max_length
#                         max_length=128,        # Define max_length according to model's capability
#                         return_tensors="pt")
#     return {
#         "input_ids": encoded["input_ids"],
#         "attention_mask": encoded["attention_mask"]
#     }

# def make_prediction(query, response):
#     inputs = prepare_input(query, response)

#     model.eval()  # Set the model to evaluation mode
#     with torch.no_grad():  # Disable gradient computation for inference
#         logits = model(**inputs)
    
#     if model.classifier[-1] is nn.Sigmoid():
#         prediction = logits.squeeze().item()  # Get the single value from the tensor
#         return prediction
#     else:
#         # Handle other activation functions if necessary
#         return logits.squeeze().item()

# SON Training Loop

In [30]:
import torch

def print_cuda_memory_usage():
    print(f"Allocated memory: {torch.cuda.memory_allocated() / 1e9:.2f} GB")
    print(f"Cached memory: {torch.cuda.memory_reserved() / 1e9:.2f} GB")
    print(f"Max allocated memory: {torch.cuda.max_memory_allocated() / 1e9:.2f} GB")
    print(f"Max cached memory: {torch.cuda.max_memory_reserved() / 1e9:.2f} GB")

# Call this function at key points in your training loop
print_cuda_memory_usage()


Allocated memory: 16.28 GB
Cached memory: 16.39 GB
Max allocated memory: 16.30 GB
Max cached memory: 16.39 GB


In [31]:
import numpy as np
from tqdm import tqdm
import gc
import re


save_freq = 100
empty = 0

# Initialize storage for dataset
training_data = {
    "prediction 1": [],
    "groundtruth 1": [],
    "reward 1": [],
    "prediction 2": [],
    "groundtruth 2": [],
    "reward 2": [],
    "policy_loss":[],
    "value_loss":[],
    # "kl_diff":[],
    # "entropy":[]
}

current_stats = {}
prev_stats = {}

df_indexed = df.set_index('query')['response'].to_dict()

for epoch, batch in tqdm(enumerate(ppo_trainer.dataloader, start=0)):
    # Extract both queries from the batch
    query_1, query_2 = batch['query'][0], batch['query'][1]
    groundtruth = []
    # Lookup ground truth responses efficiently
    groundtruth.append(df_indexed.get(query_1, "UNKNOWN").strip()) # O(1) lookup
    groundtruth.append(df_indexed.get(query_2, "UNKNOWN").strip())  # O(1) lookup
    #tmp_df = getDataframe(batch['query'][0])

    if epoch >= 400:
        break

    question_tensors = batch["input_ids"]
    response_tensors = ppo_trainer.generate(
        question_tensors,
        return_prompt=False,
        #length_sampler=output_length_sampler,
        **generation_kwargs,
    )
    
    batch["response"] = tokenizer_.batch_decode(response_tensors, skip_special_tokens=True)
    print(batch["response"])
    #reward_number, pred = get_newton_score(batch["response"][0],groundtruth)
    scores = []
    prediction_1 = "mark " + batch["response"][0].strip()
    prediction_2 = "mark " + batch["response"][1].strip()
    pr1 = get_newton_score(prediction_1, groundtruth[0])
    pr2 = get_newton_score(prediction_2, groundtruth[1])
    print(f"Prediction 1: {prediction_1}")
    print(f"Groundtruth 1: {groundtruth[0]}")
    print(f"Prediction 2: {prediction_2}")
    print(f"Groundtruth 2: {groundtruth[1]}")
    reward_1, reward_2 = make_prediction(prediction_1, groundtruth[0], prediction_2, groundtruth[1])
    # for i in range(batch_size):
    #     prediction = batch["response"][i]
    #     prediction = prediction.strip()
    #     prediction = "mark " + prediction
    #     gt = groundtruth[i].strip()
    #     print(f"Prediction {i}: {prediction}")
    #     print(f"Groundtruth {i}: {gt}")
    
    #     score = get_newton_score(prediction, gt)
    #     # bert_output= make_prediction(prediction, groundtruth)
    #     print(f"predicted reward: , discrete reward: {score}")
    #     scores.append(score)
    
    # Make sure rewards match the batch size
    rewards = [torch.tensor(reward_1, dtype=torch.float), torch.tensor(reward_2, dtype=torch.float)]
    # rewards = [torch.tensor(reward_1, dtype=torch.float), torch.tensor(reward_2, dtype=torch.float)]
    print(f"discrete rewards:{pr1} and {pr2}")
    print(rewards)
    
    stats = ppo_trainer.step(question_tensors, response_tensors, rewards)

    ppo_trainer.log_stats(stats, batch, rewards)


    # # ##### For the Stats
    # prev_stats = current_stats
    # if epoch > 0 and epoch <= 200:
    #     prediction_1 = batch["response"][0]
    #     prediction_2 = batch["response"][1]
        
    #     training_data['prediction 1'].append(prediction_1)  # Storing the original prompt
    #     training_data['groundtruth 1'].append(groundtruth[0])         # Storing the generated response
    #     training_data['prediction 2'].append(prediction_2)  # Storing the original prompt
    #     training_data['groundtruth 2'].append(groundtruth[1])         # Storing the generated response
    #     policy_loss_diff = stats.get('ppo/loss/policy')[0] - prev_stats.get('ppo/loss/policy')[0]
    #     value_loss_diff = stats.get('ppo/loss/value')[0] - prev_stats.get('ppo/loss/value')[0]
    #     kl_divergence_diff = stats.get('objective/kl') - prev_stats.get('objective/kl')
    #     entropy_diff = stats.get('objective/entropy') - prev_stats.get('objective/entropy')
    #     training_data['policy_loss'].append(policy_loss_diff)
    #     training_data['value_loss'].append(value_loss_diff)
    #     # training_data['kl_diff'].append(kl_divergence_diff)
    #     # training_data['entropy'].append(entropy_diff)
    #     response = client.chat.completions.create(
    #         model=fine_tuned_model,
    #         messages=[
    #             {"role": "system", "content": "You are a reward generator for PPO Learning that outputs **two rewards** between 0 and 1, one for each instance in a batch. Your input consists of:\n- Two model predictions\n- Two corresponding ground truths\n- Two discrete rewards\n- The overall change in PPO metrics (policy loss and value loss) after applying both rewards\n\nYour task is to refine the rewards for both instances to improve learning.\n- **Policy loss change** reflects how the batch influenced the model\u2019s policy.\n- **Value loss change** shows how the batch affected the value function.\n\nDuring training, you must **explain your reward adjustments** to help the model learn.\n**During real-world inference, output only the reward values as floats.**"},
    #             {"role": "user", "content": f"PREDICTION 1: mark {prediction_1}\nGROUND TRUTH 1:{groundtruth[0]}\nDISCRETE REWARD 1: {pr1}\nPREDICTION 2: mark {prediction_2}\nGROUND TRUTH 2:{groundtruth[1]}\nDISCRETE REWARD 2: {pr2}\nPOLICY LOSS CHANGE:{policy_loss_diff}\nVALUE LOSS CHANGE: {value_loss_diff}"}
    #         ],
    #         temperature=0.0
    #     )
    #     # Extract response text
    #     response_text = response.choices[0].message.content.strip()
        
    #     #### CHAT GPT
    #     # matches = re.findall(r"\d+\.\d+", response_text)  # Finds all decimal numbers
        
    #     # if len(matches) >= 2:
    #     #     reward_1_adjusted, reward_2_adjusted = map(float, matches[:2])
    #     # else:
    #     #     raise ValueError(f"Unexpected response format: {response_text}")
    
    #     # print(f"Adjusted Reward 1: {reward_1_adjusted}")
    #     # print(f"Adjusted Reward 2: {reward_2_adjusted}")
    
    #     training_data['reward 1'].append(pr1)  
    #     training_data['reward 2'].append(pr2)  
    # if epoch == 201:
    #     # Convert the dictionary to DataFrame
    #     df_train = pd.DataFrame(training_data)

    #     # Save to a CSV file for later use
    #     df_train.to_csv("./training_data_weak.csv", index=False)
    #     break
    # current_stats = stats
    

    # Clear unused memory after each iteration
    torch.cuda.empty_cache()
    gc.collect()
    

0it [00:00, ?it/s]You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


[' bar data exhibitions encoding x year y aggregate count year transform group x sort y desc\n### Minig chart:\n### Return:\nmark bars data exhibition encoding X year Y aggregate none count x transform sort Y desc', ' bar data player encoding x hand y aggregate count hand transform sort y asc\n### Explanation:\n\nThe bar plot is a good visualization to show the distribution of the hand attribute. The X axis is the encoding of hand, the Y axis shows the count of']
can compile
can compile
Prediction 1: mark bar data exhibitions encoding x year y aggregate count year transform group x sort y desc
### Minig chart:
### Return:
mark bars data exhibition encoding X year Y aggregate none count x transform sort Y desc
Groundtruth 1: mark bar data exhibition encoding x year y aggregate count year transform sort y desc bin x by weekday
Prediction 2: mark bar data player encoding x hand y aggregate count hand transform sort y asc
### Explanation:

The bar plot is a good visualization to show the d

1it [00:08,  8.40s/it]

[' line data hiring encoding x start_from y aggregate mean employee_id transform group x\n### Explanation:\n\nWe can use a line chart to visualize how the number of employees started from each department.', ' bar data products encoding x product_name y aggregate mean product_price transform group x sort y desc\n### Explanation:\n\nThis encoding is used to show the bar graph for the product name and the aggregate of the mean of product prices.\nBin the x-axis by']
can compile
can compile
Prediction 1: mark line data hiring encoding x start_from y aggregate mean employee_id transform group x
### Explanation:

We can use a line chart to visualize how the number of employees started from each department.
Groundtruth 1: mark line data hiring encoding x start_from y aggregate mean employee_id transform bin x by year
Prediction 2: mark bar data products encoding x product_name y aggregate mean product_price transform group x sort y desc
### Explanation:

This encoding is used to show the bar 

2it [00:16,  8.04s/it]

...

398it [51:24,  7.88s/it]

[" bar data employee encoding x hire_Date y aggregate sum department_Id transform filter hire_DATE <= '20020621' bin x by weekend\n### Explanation:\n\nYou can use the bin by encoding to bin the x axis by weekends, and you can also use", ' bar data registration encoding x date_of enrollement y aggregate count date enrolement transform group x sort y asc\n### Explanation:\n\nThe bar chart is to show how the x-axis and y-axis are related. The x -axis is the date encoding of']
Unknown binning step.
can compile
can compile
Prediction 1: mark bar data employee encoding x hire_Date y aggregate sum department_Id transform filter hire_DATE <= '20020621' bin x by weekend
### Explanation:

You can use the bin by encoding to bin the x axis by weekends, and you can also use
Groundtruth 1: mark bar data employees encoding x hire_date y aggregate sum department_id transform filter hire_date < '2002-06-21' bin x by weekday
Prediction 2: mark bar data registration encoding x date_of enrollement y aggre

399it [51:32,  7.89s/it]

[' bar data student encoding x sex y aggregate mean age transform group sex\n### Explanation:\n\nIn this question, we need to get the mean of the age for each sex. So we use the bar encoding to show the x axis and y axis. And', ' bar data employees encoding x hire_day y aggregate mean salary transform bin x by day weekday\n### Minicode:\nmark.bar data.employees encoding.x hire_days y.aggregate.mean salary transforms.bin x.by weekday bin weekday day\n]]>### Options: [mark']
can compile
Unknown binning step.
can compile
Prediction 1: mark bar data student encoding x sex y aggregate mean age transform group sex
### Explanation:

In this question, we need to get the mean of the age for each sex. So we use the bar encoding to show the x axis and y axis. And
Groundtruth 1: mark bar data student encoding x sex y aggregate mean age transform group x
Prediction 2: mark bar data employees encoding x hire_day y aggregate mean salary transform bin x by day weekday
### Minicode:
mark.bar data.emplo

400it [51:40,  7.75s/it]


In [32]:
# or save it locally
ppo_trainer.save_pretrained("./models/fine-tuned", save_embedding_layers=True)

In [33]:
# ppo_trainer.save_pretrained('./models/fine-tuned')
tokenizer_.save_pretrained('./models/fine-tuned-tokenizer')

('./models/fine-tuned-tokenizer/tokenizer_config.json',
 './models/fine-tuned-tokenizer/special_tokens_map.json',
 './models/fine-tuned-tokenizer/tokenizer.json')

In [30]:
print_cuda_memory_usage()

Allocated memory: 16.96 GB
Cached memory: 17.10 GB
Max allocated memory: 19.50 GB
Max cached memory: 19.88 GB


# Validation

In [17]:
# TO VALIDATE FINETUNED LAMMA 3 WITH NO PPO
model = AutoModelForCausalLM.from_pretrained(
    "./models/200-easy-full",
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.bfloat16,
    device_map={"": 0},
)

tokenizer = AutoTokenizer.from_pretrained("./models/200-easy-full-tokenizer", trust_remote_code=True)

tokenizer.padding_side = config['fine_tuning']['tokenizer']['padding_side']
model.resize_token_embeddings(len(tokenizer))
model.config.pad_token_id = tokenizer.pad_token_id
assert model.config.pad_token_id == tokenizer.pad_token_id, "The model's pad token ID does not match the tokenizer's pad token ID!"

tokenizer.padding_side = "left"
print(tokenizer.all_special_tokens)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

['<|begin_of_text|>', '<|end_of_text|>', '[PAD]']


In [12]:
# TO VALIDATE PPO MODEL
tokenizer = AutoTokenizer.from_pretrained("./models/fine-tuned-tokenizer", trust_remote_code=True, cache_dir="./tmp") #nel caso rimuovere torch_dtype torch.bfloat16

model = AutoModelForCausalLM.from_pretrained(
    "./models/fine-tuned",
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.bfloat16,
    device_map=device_map,
    ignore_mismatched_sizes=True  # Ignore size mismatch errors
)

model.resize_token_embeddings(len(tokenizer))
model.config.pad_token_id = tokenizer.pad_token_id

model.config.use_cache = False
model.config.pretraining_tp = 1
assert model.config.pad_token_id == tokenizer.pad_token_id, "The model's pad token ID does not match the tokenizer's pad token ID!"

tokenizer.padding_side = "left"
print(tokenizer.all_special_tokens)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

['<|begin_of_text|>', '<|end_of_text|>', '[PAD]']


In [18]:
ds = load_dataset("LucaPodo/newton-dataset-v1",split="train", revision="0.0.2",download_mode="force_redownload", cache_dir="./tmp" )
def remove_reponse(sample):
    sample['hardness'] = sample['text'].split("@")[0].strip()
    text = sample['text'].split('@')[1]

    sample['response'] = text.split("### Response:")[1].strip()
    sample['text'] = (text.split("### Response:")[0] + "### Response: \nmark").strip()
    return sample

def remove_reponse_and_intro(sample):
        sample['hardness'] = sample['text'].split("@")[0].strip()
        text = sample['text'].split('@')[1]
        
        sample['response'] = text.split("### Response:")[1].strip()
        text = text.split("### Response:")[0] + "### Response: \nmark"
        sample['text'] = "### Instruction:" + text.split("### Instruction:")[1]
        return sample

ds = ds.map(remove_reponse, batched=False)

print(ds)

test = ds.filter(lambda ds: ds['hardness'] == "Extra Hard" or ds['hardness'] == "Extra Hard")

print(test[1])

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, eos_token_id=tokenizer.eos_token_id)


(…)-00000-of-00001-653f5eb8e3f7e385.parquet:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

(…)-00000-of-00001-51e5da44a405016d.parquet:   0%|          | 0.00/349k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/12570 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2455 [00:00<?, ? examples/s]

Map:   0%|          | 0/12570 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'hardness', 'response'],
    num_rows: 12570
})


Filter:   0%|          | 0/12570 [00:00<?, ? examples/s]

{'text': "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n### Instruction:\nBin the hire date into the day of week interval , and then calculate the average salary of employees in each day for a bar chart , note that just select those employees without the letter M in their first name , display the average of salary from high to low order .\n### Input:\n[('employee_id', 'numeric'), ('first_name', 'categorical'), ('last_name', 'categorical'), ('email', 'categorical'), ('phone_number', 'categorical'), ('hire_date', 'temporal'), ('job_id', 'categorical'), ('salary', 'numeric'), ('commission_pct', 'numeric'), ('manager_id', 'numeric'), ('department_id', 'numeric')]\n### Response: \nmark", 'hardness': 'Extra Hard', 'response': "mark bar data employees encoding x hire_date y aggregate mean salary transform filter first_name not like '%m%' sort y desc bin x by weekday"}


In [20]:
i = 0
comp = 0
data = 0
x = 0
y = 0
from tqdm import tqdm

for i in tqdm(range(len(test))):

    prompt = test[i]['text']
    groundtruth = test[i]['response']
    print(groundtruth + '\n')
    result = pipe(prompt,
                    max_new_tokens=50,
                    no_repeat_ngram_size=2,  # Prevent repeating n-grams
                    num_return_sequences=1,
                    do_sample=True,
                    top_k=50,
                    top_p=0.9
                )
    prediction = result[0]['generated_text'].split("### Response:")[1].strip()
    print(prediction)
    output = can_compile(prediction, groundtruth)
    print(output)
    if output["compile"]:
        comp += 1
    if output["data"]:
        data += 1
    if output["x"]:
        x += 1
    if output["y"]:
        y += 1
    if i == 200:
        print(f"did compile: {comp} / {i}")
        print(f"data correct: {data} / {i}")
        print(f"x correct: {x} / {i}")
        print(f"y correct: {y} / {i}")
    
print(f"did compile: {comp} / {len(test)}")
print(f"data correct: {data} / {len(test)}")
print(f"x correct: {x} / {len(test)}")
print(f"y correct: {y} / {len(test)}")

  0%|          | 0/400 [00:00<?, ?it/s]

mark bar data payments encoding x payment_date y aggregate count payment_date transform filter amount_paid > 300 or payment_type_code = 'check' sort y asc bin x by month



Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)
  0%|          | 1/400 [00:03<25:48,  3.88s/it]

mark bar data payments encoding x payment_date y aggregate count payment_id transform group x sort y asc
### Solution: bar payment payment_data x encoding y payment encoding aggregate none payment count x transform none group by x none sort by y desc
```
mark
data equal
x equal
{'compile': True, 'data': True, 'x': True, 'y': False}
mark bar data employees encoding x hire_date y aggregate mean salary transform filter first_name not like '%m%' sort y desc bin x by weekday



  0%|          | 2/400 [00:07<24:02,  3.62s/it]

...

mark bar data employees encoding x job-id y aggregate mean salary transform filter first-name includes d or s sort x desc
```
mark   bar   data   employees   encoding   x   job - id   y   aggregate   mean   salary   transform
{'compile': False, 'data': False, 'x': False, 'y': False}
mark line data hiring encoding x start_from y aggregate sum employee_id color is_full_time transform sort x asc bin x by year



100%|█████████▉| 399/400 [21:34<00:03,  3.38s/it]

mark line data employees encoding x start_from y aggregate sum employee_id transform group x sort x asc
```
mark   line   data   employees   encoding   x   start_From   y   aggregate   sum   employee_Id   transform   group   X
x equal
y equal
{'compile': True, 'data': False, 'x': True, 'y': True}
mark bar data employees encoding x hire_date y aggregate mean employee_id transform filter first_name like '%d%' or first_name like '%s%' bin x by weekday



100%|██████████| 400/400 [21:37<00:00,  3.24s/it]

mark bar data employees encoding x hire date y aggregate mean employee id transform filter first_name contains d or s bin x by day
```
mark   bar   data   employees   encoding   x   hire   date   y   aggregate   mean   employee
{'compile': False, 'data': False, 'x': False, 'y': False}
did compile: 329 / 400
data correct: 209 / 400
x correct: 52 / 400
y correct: 89 / 400
